In [84]:
from pandas import DataFrame
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from rfpimp import permutation_importances
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [218]:
# 提取整列,参数：当前的数据路径和需要提取的列号
def column(path,i):
    df_li = path.values.tolist()
    result = []
    for s_li in df_li:
        result.append(s_li[i])
    return result

In [219]:
#标准化
def normalization(df,min,max):
    sheet = []
    for i in range(0,7397):
        lie = []
        for j in range(0,32):
            if((j == 1)or(j == 6)or(j == 9)):
                df[i][j] = (df[i][j] - min[j]) / (max[j] - min[j])
            else:
                df[i][j] = (max[j] - df[i][j])/(max[j] - min[j])
            lie.append(df[i][j])
        sheet.append(lie)
    return sheet

In [220]:
def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

In [221]:
df = pd.read_excel('T5通勤时间.xlsx')
# 最大值
max = []
# 这里数字需要修改
for i in range(1,33):
    a = column(df,i)
    max.append(np.max(a))
# 最小值
min = []
for i in range(1,33):
    a = column(df,i)
    min.append(np.min(a))


In [222]:
sheet1 = []
name = ['社会治安', '交通秩序', '消防安全隐患', '紧急避难场所', 
        '内涝积水', '自然灾害应对', '安全事故应对', '标志性建筑（特色建筑）',
        '文化设施', '历史街区保护', '历史建筑与传统民居的修复和利用', '游客吸引力',
       '房价的可接受程度', '房租的可接受程度', '住房租赁市场的规范程度', '外来人口的友好性',
       '弱势群体的关爱性', '最低生活保障水平', '保障性住房建设', '棚户区及城中村改造', 
       '人才引进政策', '工作机会', '市场环境', '科技创新环境',
       '贷款方便', '小区垃圾分类', '物业管理', '街道卫生',
       '窨井盖维护', '立杆管理', '路灯管理维护', '机动车停放管理']
name = np.array(name)


In [223]:
for num in range(0, 7397):
    data = df.iloc[num].values 
    lie = data[1:33]
    sheet1.append(lie)

In [224]:
data = np.array(sheet1)
data.shape

(7397, 32)

In [225]:
data = normalization(data,min,max)


In [226]:
data = np.array(data)
data = pd.DataFrame(data, columns = name)


target = column(df,33)
target = np.array(target)
rf = RandomForestRegressor(n_estimators = 100,n_jobs = -1,oob_score = True,bootstrap = True,random_state = 42)
rf.fit(data, target)

C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=True, random_state=42, verbose=0,
                      warm_start=False)

In [227]:
print('GINI')
gini=dict(zip(list(name),list(rf.feature_importances_)))
print(gini)

GINI
{'社会治安': 0.0005909845082361845, '交通秩序': 0.027372898424669365, '消防安全隐患': 0.006016902602996257, '紧急避难场所': 0.006134352292664414, '内涝积水': 0.001282804384355501, '自然灾害应对': 0.004449883566253014, '安全事故应对': 0.26211233245004717, '标志性建筑（特色建筑）': 0.0076354741370101694, '文化设施': 0.001768745799404675, '历史街区保护': 0.5030241317341314, '历史建筑与传统民居的修复和利用': 0.011043564307333466, '游客吸引力': 0.006335938562576878, '房价的可接受程度': 0.022699972328040576, '房租的可接受程度': 0.009156490708497667, '住房租赁市场的规范程度': 0.004840594287265055, '外来人口的友好性': 0.010634400664090304, '弱势群体的关爱性': 0.0029603059833106927, '最低生活保障水平': 0.0028876163369540345, '保障性住房建设': 0.0075133656325793415, '棚户区及城中村改造': 0.0012266776776544582, '人才引进政策': 0.0018169029008310467, '工作机会': 0.002176853146767889, '市场环境': 0.0015470886253050405, '科技创新环境': 0.0009275929863184173, '贷款方便': 0.0008820219804305142, '小区垃圾分类': 0.003898886396943517, '物业管理': 0.024476701135867664, '街道卫生': 0.01160650026915469, '窨井盖维护': 0.03101250926757974, '立杆管理': 0.0013924260750151282, '路灯管理维护': 0.00255

C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [228]:
target = pd.DataFrame(target)
#target

In [229]:
print('OOB')


perm_imp_rfpimp = permutation_importances(rf, data, target, r2)
perm_imp_rfpimp.reset_index(drop = False, inplace = True)
print(perm_imp_rfpimp)


OOB


C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\s

            Feature  Importance
0            安全事故应对    0.220190
1            历史街区保护    0.219684
2              交通秩序    0.053086
3              物业管理    0.037971
4           机动车停放管理    0.032427
5          房价的可接受程度    0.029381
6             窨井盖维护    0.016462
7          外来人口的友好性    0.016249
8              街道卫生    0.014518
9   历史建筑与传统民居的修复和利用    0.014329
10      标志性建筑（特色建筑）    0.010661
11            游客吸引力    0.008723
12      住房租赁市场的规范程度    0.007151
13          保障性住房建设    0.005831
14           自然灾害应对    0.005168
15           消防安全隐患    0.004853
16           紧急避难场所    0.004374
17         房租的可接受程度    0.003992
18           小区垃圾分类    0.003764
19           路灯管理维护    0.002422
20         最低生活保障水平    0.001941
21         弱势群体的关爱性    0.001585
22             文化设施    0.000831
23           人才引进政策    0.000831
24             工作机会    0.000819
25             内涝积水    0.000772
26           科技创新环境    0.000424
27             市场环境    0.000400
28             社会治安    0.000393
29             贷款方便    0.000255
30      

C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [230]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.6, random_state=1)
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)
print('MSE train: %.4f, test: %.4f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.4f, test: %.4f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Xiaoling Zhang\.conda\envs\test_env\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


MSE train: 148.4518, test: 152.5364
R^2 train: 0.5404, test: 0.5271
